In [ ]:
from flask import Flask, request, jsonify, render_template
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import nltk
from nltk.corpus import stopwords
import threading
import webbrowser

#nltk.download('stopwords')             #one time download and comment out after that
stop_words = set(stopwords.words('english'))


In [ ]:
df = pd.read_csv('D:\Spam-Detection-Using-Naive-Bayes\sms.tsv', sep='\t', names=['label', 'message'])

def preprocess(msg):
    msg = msg.lower()
    msg = ''.join([ch for ch in msg if ch not in string.punctuation])
    words = msg.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['cleaned'] = df['message'].apply(preprocess)


<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Admin\AppData\Local\Temp\ipykernel_30424\4105646008.py:2: SyntaxWarning: invalid escape sequence '\S'
  df = pd.read_csv('D:\Spam-Detection-Using-Naive-Bayes\sms.tsv', sep='\t', names=['label', 'message'])


In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['cleaned']).toarray()
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = GaussianNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print(classification_report(y_test, y_pred))


Model Accuracy: 89.89%
              precision    recall  f1-score   support

         ham       0.98      0.90      0.94      1448
        spam       0.58      0.91      0.71       224

    accuracy                           0.90      1672
   macro avg       0.78      0.90      0.82      1672
weighted avg       0.93      0.90      0.91      1672



In [ ]:
app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    input_msg = data.get("message", "")
    
    if not input_msg.strip():
        return jsonify({"error": "Message cannot be empty"}), 400

    cleaned_msg = preprocess(input_msg)
    vectorized_msg = vectorizer.transform([cleaned_msg]).toarray()
    prediction = model.predict(vectorized_msg)[0]

    return jsonify({
        "prediction": prediction,
        "accuracy": f"{accuracy * 100:.2f}%"
    })



In [ ]:
def run_flask():
    app.run(port=5000)

def open_browser():
    webbrowser.open("http://127.0.0.1:5000")

threading.Timer(1, open_browser).start()
run_flask()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Jun/2025 17:27:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2025 17:27:26] "POST /predict HTTP/1.1" 200 -
